In [11]:
import json
import numpy as np
from keras import layers
from keras import models
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [12]:
import pickle
import pandas as pd
df=pd.read_pickle('./output/DataSetAudios.pkl')
number_classes = len(set(df['name']))
number_classes
y_names=df['name']
type(y_names)

pandas.core.series.Series

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_names)
y=np.vstack(le.transform(y_names))

In [14]:
type(y)

numpy.ndarray

In [15]:
#y=np.vstack(df['name'])
X=np.vstack(df['features'])
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)

'''validation_samples = 10
x_val = X_train[:validation_samples]
partial_x_train = X_train[validation_samples:]
y_val = y_train[:validation_samples]
partial_y_train = y_train[validation_samples:]'''

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



(1108, 2176) (278, 2176) (1108, 1) (278, 1)


In [16]:
y_train

array([[ 2],
       [13],
       [ 8],
       ...,
       [ 1],
       [14],
       [15]])

In [20]:
inshape=(X_train.shape[1],)
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=inshape))
model.add(layers.Dense(256, activation='relu'))

model.add(BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(number_classes, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,
          y_train,
          epochs=60,
          batch_size=50,
          validation_data=(X_test, y_test))

results = model.evaluate(X_test, y_test)
print("")
print(results)
print("\n")

predictions = model.predict(X_test)

#predictions = nn_to_labels(predictions)
#confusion = confusion_matrix(y_test, predictions)

print(model)

Train on 1108 samples, validate on 278 samples
Epoch 1/60
1108/1108 [==============================] - 1s 666us/step - loss: 2.4510 - accuracy: 0.2951 - val_loss: 2.5818 - val_accuracy: 0.2914
Epoch 2/60
1108/1108 [==============================] - 0s 360us/step - loss: 1.9399 - accuracy: 0.4549 - val_loss: 2.0197 - val_accuracy: 0.3777
Epoch 3/60
1108/1108 [==============================] - 0s 359us/step - loss: 1.7413 - accuracy: 0.5108 - val_loss: 1.9852 - val_accuracy: 0.4388
Epoch 4/60
1108/1108 [==============================] - 0s 365us/step - loss: 1.5660 - accuracy: 0.5496 - val_loss: 1.7364 - val_accuracy: 0.5000
Epoch 5/60
1108/1108 [==============================] - 0s 373us/step - loss: 1.3890 - accuracy: 0.6155 - val_loss: 1.6543 - val_accuracy: 0.5360
Epoch 6/60
1108/1108 [==============================] - 0s 361us/step - loss: 1.2481 - accuracy: 0.6480 - val_loss: 1.5603 - val_accuracy: 0.5791
Epoch 7/60
1108/1108 [==============================] - 0s 366us/step - loss:

1108/1108 [==============================] - 0s 404us/step - loss: 0.5662 - accuracy: 0.8141 - val_loss: 1.0312 - val_accuracy: 0.6691
Epoch 57/60
1108/1108 [==============================] - 0s 424us/step - loss: 0.5462 - accuracy: 0.8240 - val_loss: 1.2023 - val_accuracy: 0.6547
Epoch 58/60
1108/1108 [==============================] - 0s 385us/step - loss: 0.5235 - accuracy: 0.8366 - val_loss: 0.9619 - val_accuracy: 0.7266
Epoch 59/60
1108/1108 [==============================] - 1s 470us/step - loss: 0.4892 - accuracy: 0.8520 - val_loss: 0.9638 - val_accuracy: 0.7338
Epoch 60/60
278/278 [==============================] - 0s 106us/step

[3.9918526839866915, 0.6007194519042969]




In [21]:
from joblib import dump, load
dump(model, f'./models/primer_nn.joblib')

['./models/primer_nn.joblib']

In [25]:
#le.inverse_transform
#model.save('./models/primerito.hf5')
#model.load_weights('./models/weights_primerito.hf5')

In [26]:
import time
moment=time.localtime()
name='./models/Model_{}_{}-{}-{}.h5'.format(model.history.history["val_accuracy"][-1],moment[2],moment[3],moment[4])
model.save(name)

In [27]:
import json

model_json = model.to_json()
with open(name+'.json', "w") as json_file:
    json.dump(model_json, json_file)

model.save_weights(name+'.h5')

In [40]:
from keras.models import model_from_json
with open('./models/Model_0.6007194519042969_16-20-44.h5.json','r') as f:
    model_json = json.load(f)

model = model_from_json(model_json)
model.load_weights('./models/Model_0.6007194519042969_16-20-44.h5.h5')
print('Model loaded')

Model loaded


In [43]:
X_test=X_train[1:]
predictions = model.predict(X_test)

In [44]:
predictions.shape

(1107, 19)

In [47]:
predictions[0]

array([0.02725567, 0.08089689, 0.1410461 , 0.15035963, 0.05367814,
       0.13506287, 0.03406662, 0.01360706, 0.04970345, 0.09857716,
       0.00040458, 0.00173293, 0.03785447, 0.13278052, 0.00190351,
       0.01350835, 0.0188138 , 0.00786899, 0.00087919], dtype=float32)